In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.models import create_cnn, create_mlp, create_multi_model

config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

def memory_creator(new_frame, memory, length=4, interval=2):
    memory.append(new_frame)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=2)

filename = 'lap_6_2020_01_24'

In [2]:
df = reader.read_telemetry_as_csv(filename + '.csv')
telemetry = collector.collect_numeric_inputs(df)
diffs = collector.collect_expert_labels(df)

frames = reader.read_training_video(filename + '.avi')
resized_frames_np = transformer.resize_video_for_training(frames)
    
print(resized_frames_np.shape)
print(telemetry.shape)
print(telemetry.columns.values)
print(diffs.shape)
print(diffs.columns.values)

telemetry_np = telemetry.to_numpy()
diffs_np = diffs.to_numpy()

frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(resized_frames_np, telemetry_np, diffs_np, test_size=0.2)

(6578, 40, 60, 3)
(6578, 4)
['cg' 'cs' 'ct' 'cb']
(6578, 4)
['d_gear' 'd_steering' 'd_throttle' 'd_braking']


In [3]:
def prep_mem_datas(telemetry, diffs, resized_frames_np, length, interval):
    mem_telemetry = telemetry
    mem_diffs = diffs

    memory = []
    mem_frames = []
    for frame in resized_frames_np:
        mem_frame = memory_creator(frame, memory, length=length, interval=interval)

        if mem_frame is not None:
            mem_frames.append(mem_frame)

    mem_frames_np = np.array(mem_frames)

    len_diff = mem_telemetry.shape[0] - mem_frames_np.shape[0]
    if len_diff > 0:
        mem_telemetry = mem_telemetry.iloc[len_diff:]
        mem_diffs = mem_diffs.iloc[len_diff:]
    
    mem_telemetry_np = mem_telemetry.to_numpy()
    mem_diffs_np = mem_diffs.to_numpy()

    return mem_frames_np, mem_telemetry_np, mem_diffs_np

In [11]:
a = [1, 2]
b = [5, 5]

[np.mean(i) for i in zip(a, b)]

[3.0, 3.5]

In [13]:
epochs = 6
batch_size = 32
verbose = 0

experiments = [(1, 1), (4, 4), (4, 8), (4, 12)]
datas_list = []

for experiment in experiments:
    datas_list.append(prep_mem_datas(telemetry, diffs, resized_frames_np, *experiment))

for data in datas_list:
    frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(*data, test_size=0.2)
    
    print("Memory: N={}, M={}".format(*experiments[datas_list.index(data)]))
    losses = []
    val_losses = []
    for i in range(0, 6):
        mlp = create_mlp()
        cnn = create_cnn(input_shape=frames_train[0].shape)
        multi = create_multi_model(mlp, cnn)
    
        hist = multi.fit([telemetry_train, frames_train], diffs_train,
                        validation_data=([telemetry_test, frames_test], diffs_test),
                        epochs=epochs, batch_size=batch_size, verbose=verbose)
        
        losses.append(hist.history['loss'])
        val_losses.append(hist.history['val_loss'])
    
    mean_losses = [np.mean(i) for i in zip(*losses)]
    mean_val_losses = [np.mean(i) for i in zip(*val_losses)]
    
    print("Mean loss per epoch: {}".format(mean_losses))
    print("Mean validation loss per epoch: {}".format(mean_val_losses))
    print("\n")

Memory: N=1, M=1
Mean loss per epoch: [0.3959003766592987, 0.23230686702516676, 0.1735980800546146, 0.13081118310606404, 0.10135769059209714, 0.08043629454488228]
Mean validation loss per epoch: [[0.34168068620017705, 0.20253780720375283, 0.15565336168559954, 0.12239821666957126, 0.09752560225968486, 0.07682762072283204], [0.464724750176259, 0.28827546060606357, 0.2246788481327573, 0.15937996320412304, 0.10932236612522751, 0.08379596070231073], [0.3551328842209936, 0.20656707641608088, 0.15156283835320816, 0.1133516939793735, 0.09132415447872612, 0.07496879693277886], [0.41498397349042615, 0.24043854762963224, 0.16953661708222828, 0.12006416542760685, 0.09363240412220851, 0.07344190563676657], [0.4939081247475427, 0.23773198878071872, 0.16406372580033027, 0.13125907411959542, 0.1120987827477234, 0.09567824848270924], [0.30497184112039366, 0.21829032151475236, 0.17609308927356404, 0.13841398523611426, 0.10424283381901236, 0.07790523479189632]]


Memory: N=4, M=4


/home/martin/miniconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


KeyboardInterrupt: 

In [6]:
mlp = create_mlp()
cnn = create_cnn(input_shape=frames_train[0].shape)
multi = create_multi_model(mlp, cnn)
#multi.summary()

multi.fit([telemetry_train, frames_train], diffs_train,
                validation_data=([telemetry_test, frames_test], diffs_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)

Train on 5233 samples, validate on 1309 samples
Epoch 1/6
5233/5233 [==============================] - 4s 778us/sample - loss: 0.3701 - val_loss: 0.2374
Epoch 2/6
5233/5233 [==============================] - 3s 577us/sample - loss: 0.2323 - val_loss: 0.1610
Epoch 3/6
5233/5233 [==============================] - 2s 462us/sample - loss: 0.1624 - val_loss: 0.0921
Epoch 4/6
5233/5233 [==============================] - 3s 577us/sample - loss: 0.1165 - val_loss: 0.0619
Epoch 5/6
5233/5233 [==============================] - 3s 567us/sample - loss: 0.0924 - val_loss: 0.0496
Epoch 6/6
5233/5233 [==============================] - 3s 554us/sample - loss: 0.0750 - val_loss: 0.0428
